In [1]:
from sklearn.cluster import KMeans
import pandas as pd 
import numpy as np

def main():
    NUMBER_OF_CLUSTERS = 5

    df = pd.read_hdf('data/latent_spaces.h5')
    latent_space = np.stack(df['latent_space'].values)

    # Creating a KMeans model with chosen number of clusters
    kmeans = KMeans(n_clusters=NUMBER_OF_CLUSTERS, random_state=0, n_init=5)

    # Fitting the model to your data
    kmeans.fit(latent_space)

    # Getting the cluster assignments for each image
    df['cluster'] = kmeans.labels_

    # saving data
    df.to_hdf('data/latent_spaces.h5', key='df_items', mode='w')
    

if __name__ == "__main__":
    main()

C:\Users\amish\AppData\Local\Temp\ipykernel_1916\2767806870.py:15: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans.fit(latent_space)
C:\Users\amish\AppData\Local\Temp\ipykernel_1916\2767806870.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['latent_space'], dtype='object')]

  df.to_hdf('data/latent_spaces.h5', key='df_items', mode='w')


In [2]:
from tensorflow.keras.models import Model, load_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import cv2
import os
import re

def get_images_and_filenames(path):
    image_list = []
    filename_list = []

    for filename in os.listdir(path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            img = img / 255.0
            image_list.append(img)
            filename_list.append(filename)
            
    filename_list = [int(re.findall(r'\d+', s)[0]) for s in filename_list]

    X_train = np.array(image_list, dtype=np.float32)
    
    return X_train, filename_list

def create_encoder(path):
    autoencoder = load_model(path)
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dropout_4').output)
        
    return encoder

def get_latent_spaces(data, encoder):
    latent_spaces = []
    
    for img in data:
        img = np.expand_dims(img, axis=0)
        latent_space = encoder.predict(img)
        flattened_latent_space = np.reshape(latent_space, (-1))
        latent_spaces.append(flattened_latent_space)

    return np.array(latent_spaces)

def scale_latent_spaces(latent_spaces):
    scaler = StandardScaler()
    return scaler.fit_transform(latent_spaces)

def use_pca_latent_spaces(latent_spaces, n_components=77):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(latent_spaces)

def load_model_and_encoder(model_path):
    autoencoder = load_model(model_path)
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dropout_4').output)
    return encoder

def test_model(image_path, model_path, hdf5_path):
    X_test, file_names = get_images_and_filenames(image_path)
    encoder = load_model_and_encoder(model_path)
    latent_spaces = get_latent_spaces(X_test, encoder)
    latent_spaces_normalized = scale_latent_spaces(latent_spaces)
    latent_spaces_pca = use_pca_latent_spaces(latent_spaces_normalized)
    
    df = pd.read_hdf(hdf5_path)
    latent_space = np.stack(df['latent_space'].values)

    kmeans = KMeans(n_clusters=5, random_state=0, n_init=5)
    kmeans.fit(latent_space)

    cluster_labels = kmeans.predict(latent_spaces_pca)
    
    return file_names, cluster_labels

def main():
    IMAGE_PATH = "images/segmented_images/"
    MODEL_PATH = "models/autoencoder.h5"
    HDF5_PATH = "data/latent_spaces.h5"
    
    file_names, cluster_labels = test_model(IMAGE_PATH, MODEL_PATH, HDF5_PATH)
    
    for file_name, cluster_label in zip(file_names, cluster_labels):
        print(f"Image: {file_name} is in cluster {cluster_label}")

if __name__ == "__main__":
    main()


C:\Users\amish\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━

C:\Users\amish\AppData\Local\Temp\ipykernel_23272\2161362670.py:72: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans.fit(latent_space)
